In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score, make_scorer

In [6]:
df = pd.read_csv('used_cars_clean.csv')
x = df.drop(columns=['price'], axis=1)
y = df['price']

In [7]:
y_log = np.log1p(y)

x_train, x_test, y_train, y_test = train_test_split(x, y_log, test_size=0.2, random_state=42)

In [8]:
model = HistGradientBoostingRegressor(
    max_depth=10,
    learning_rate=0.1,
    max_iter=300,
    random_state=42
)

model.fit(x_train, y_train)

ValueError: could not convert string to float: 'Hyundai'

In [ ]:
y_pred_log = model.predict(x_test)
y_pred = np.expm1(y_pred_log)
y_test_exp = np.expm1(y_test)

mae = mean_absolute_error(y_test_exp, y_pred)
rmse = root_mean_squared_error(y_test_exp, y_pred)
r2 = r2_score(y_test_exp, y_pred)

print(f"Mean Absolute Error: {mae}")
print(f"Root Mean Squared Error: {rmse}")
print(f"R^2 Score: {r2}")